In [1]:
import torch
import numpy as np
from torch import nn
from torch import optim

from torch.utils.data import DataLoader, Dataset

In [2]:
# super parameters

class params:
    FeaLen = 50
    Epoch = 10
    BatchSize = 64
    EmbedDim = 64
    CateNum = 11
    CateFeaLen = 10

# define model

In [3]:
class WideAndDeep(nn.Module):
    def __init__(self, featureNum, catfeaNum, catNum, embedDim):
        super().__init__()
        self.fc1 = nn.Linear(featureNum, 2)
        self.embed = nn.Embedding(num_embeddings=catNum, embedding_dim=embedDim)
        self.fc2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(catfeaNum*embedDim,500))
        self.fc3 = nn.Linear(500, 1)
    
    def forward(self, x, catx):
        # wide(linear) part
        pred = self.fc1(x)
        # deep part
        out = self.embed(catx)
        out = self.fc2(out)
        out = self.fc3(out)
        # join part
        pred = pred + out
        return pred

# fake data

In [4]:
class FakeData(Dataset):
    def __init__(self, x, catx, y):
        self.x = x
        self.catx = catx
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, i):
        data = self.x[i]
        catdata = self.catx[i]
        label = self.y[i]
        
        return data, catdata, label

In [5]:
x = np.random.rand(500,50)
y = [1 if v>0.5 else 0 for v in np.random.rand(500)]

In [6]:
catx = np.random.rand(500, 10)

catx = catx*10

catx = catx.round()

catx = catx.astype(np.int64)

In [7]:
x = x.astype(np.float32)
y = np.array(y, np.int64)

In [8]:
dataset = FakeData(x, catx, y)
dataloader = DataLoader(dataset=dataset,
                        batch_size=params.BatchSize)

# training

In [9]:
def train_model(params, dataloader):
    # init model
    model = WideAndDeep(params.FeaLen, 
                        params.CateFeaLen,
                        params.CateNum, 
                        params.EmbedDim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch_i in range(params.Epoch):
        epoch_loss = 0
        for x,catx,y in dataloader:
            optimizer.zero_grad()

            outputs = model.forward(x, catx)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        epoch_loss /= 20
        print("epoch {} loss {}".format(epoch_i, epoch_loss))

In [10]:
train_model(params, dataloader)

epoch 0 loss 0.2767000705003738
epoch 1 loss 0.2763074904680252
epoch 2 loss 0.2759425610303879
epoch 3 loss 0.2756965756416321
epoch 4 loss 0.27551392614841463
epoch 5 loss 0.275350022315979
epoch 6 loss 0.2751928001642227
epoch 7 loss 0.27504209578037264
epoch 8 loss 0.2748973101377487
epoch 9 loss 0.2747560262680054
